## **0. Importar archivos** df_users_review_curado_sentiments

In [ ]:
import pandas as pd
import os
from typing import List
import pandas as pd

In [ ]:
# Directorio de entrada
directorio_entrada = "bases/"

# Rutas completas de los archivos Parquet
ruta_archivo_parquet_f1 = os.path.join(directorio_entrada, "df_f1.parquet")
ruta_archivo_parquet_f2_f3 = os.path.join(directorio_entrada, "df_f2_f3.parquet")
ruta_archivo_parquet_f4_f5 = os.path.join(directorio_entrada, "df_f4_f5.parquet")

# Importar los DataFrames desde archivos Parquet
df_f1 = pd.read_parquet(ruta_archivo_parquet_f1)
df_f2_f3 = pd.read_parquet(ruta_archivo_parquet_f2_f3)
df_f4_f5 = pd.read_parquet(ruta_archivo_parquet_f4_f5)

# Mostrar información sobre los DataFrames recién importados
print("Información del DataFrame df_f1:")
print(df_f1.info())

print("\nInformación del DataFrame df_f2_f3:")
print(df_f2_f3.info())

print("\nInformación del DataFrame df_f4_f5:")
print(df_f4_f5.info())

## **1. Función PlayTimeGenre**

In [ ]:

def playtime_genre(genero: str):
    # Filtra el DataFrame por el género específico
    filtered_df = df_f1[df_f1['genres'] == genero]

    # Agrupa por año y calcula las horas jugadas sumando
    grouped_df = filtered_df.groupby('year_release')['playtime_forever'].sum().reset_index()

    # Encuentra el año con más horas jugadas
    max_year = grouped_df.loc[grouped_df['playtime_forever'].idxmax()]['year_release']

    return {"Año de lanzamiento con más horas jugadas para {}: {}".format(genero, max_year)}

## **2. Función UserForGenre (FUNCIONA. REVISADO)**

In [ ]:
def UserForGenre(genero: str):
    # Filtrar el DataFrame por el género específico
    df_genre = df_f2_f3[df_f2_f3['genres'] == genero]

    # Agrupar por usuario y año, sumar las horas jugadas
    grouped_df = df_genre.groupby(['user_id', 'year_release'])['playtime_forever'].sum().reset_index()

    # Encontrar al usuario con más horas jugadas
    max_user = grouped_df.loc[grouped_df['playtime_forever'].idxmax()]['user_id']

    # Filtrar el DataFrame original para obtener las horas jugadas por año del usuario máximo
    max_user_df = df_f2_f3[df_f2_f3['user_id'] == max_user]
    max_user_hours_by_year = max_user_df.groupby('year_release')['playtime_forever'].sum().reset_index()

    # Crear la lista de la acumulación de horas jugadas por año
    horas_jugadas_por_anio = [{"Año": int(anio), "Horas": int(horas)} for anio, horas in zip(max_user_hours_by_year['year_release'], max_user_hours_by_year['playtime_forever'])]

    # Crear el diccionario de retorno
    resultado = {"Usuario con más horas jugadas para Género " + genero: max_user, "Horas jugadas": horas_jugadas_por_anio}

    return resultado




## **3. Función users recommend** (FUNCIONA)

In [ ]:
def UsersRecommend(year):
    # Filtrar por año
    df_filtered = df_f2_f3[df_f2_f3['year_review'] == year]

    # Filtrar por recomendaciones positivas/neutrales
    df_filtered = df_filtered[(df_filtered['recommend'] == True) & (df_filtered['Sentiment_Score'].isin([1, 2]))]

    # Obtener el top 3 de juegos más recomendados
    top_games = df_filtered.groupby('item_name')['recommend'].count().sort_values(ascending=False).head(3)

    return top_games


## **4. Función UsersWorstDeveloper**


In [ ]:
def UsersWorstDeveloper(year:int):
    # Filtrar por el año deseado en la columna 'year_review'
    df_year = df_f4_f5[df_f4_f5['year_review'] == year]

    # Agrupar por desarrolladora y calcular las estadísticas
    grouped_stats = df_year.groupby('developer').agg({
        'recommend': lambda x: (x == False).sum(),
        'Sentiment_Score': lambda x: (x == 0).sum()
    }).reset_index()

    # Ordenar por la suma de juegos no recomendados en orden descendente
    sorted_stats = grouped_stats.sort_values(by=['recommend'], ascending=[False])

    # Seleccionar las top 3 desarrolladoras
    top3_developers = sorted_stats.head(3)

    # Crear el formato de retorno
    result = []
    for i, row in enumerate(top3_developers.iterrows(), 1):
        developer = row[1]['developer']
        result.append({f"Puesto {i}": developer})

    return result



## **5. FUNCION SENTIMENT_ANALYSIS**

In [ ]:
def sentiment_analysis(developer):
    # Filtrar el dataframe por la desarrolladora especificada
    developer_df = df_f4_f5[df_f4_f5['developer'] == developer]

    # Sumar los valores de Sentiment_Score para cada categoría
    negative_sum = developer_df[developer_df['Sentiment_Score'] == 0].shape[0]
    neutral_sum = developer_df[developer_df['Sentiment_Score'] == 1].shape[0]
    positive_sum = developer_df[developer_df['Sentiment_Score'] == 2].shape[0]

    # Crear el diccionario de salida
    result_dict = {
        developer: {
            'Negative': negative_sum,
            'Neutral': neutral_sum,
            'Positive': positive_sum
        }
    }

    return result_dict